## Agents on Kubeflow

In this tutorial we will be training a reinforcement learning agent from the [tensorflow/agents](https://github.com/tensorflow/agents) project on Kubernetes using [Kubeflow](https://github.com/google/kubeflow).

The task the agent will be learning to perform is to operate a Kuka Robotics arm simulated in the OpenAI Gym Bullet Physics 'KukaBulletEnv-v0' environment. Feel free to [skip to the end](http://localhost:8888/notebooks/kubeflow-rl/apps/agents_ppo/demo.ipynb#Rendering-the-model) to see what this will look like!

### Setup

We need to create a Google Cloud Storage bucket to store job logs as well as a unique subdirectory of that bucket to store logs for this particular run. With the following we first create the GCS bucket then generate the path of a log dir to use in a later step.

Set the variables below to a project a bucket suitable for your use

In [53]:
# GCP project to use
#
# settings for project: kubeflow-rl
PROJECT="kubeflow-dev"
# Bucket to use
BUCKET=PROJECT+"-jlewi"
# K8s cluster to use
CLUSTER="gpu-cluster"
ZONE="us-central1-a"
NAMESPACE="jlewi-rl"
# Root directory for the kubeflow-rl repository
ROOT_DIR = "/home/jovyan/git_kubeflow-rl"
SECRET_NAME = "kubeflow-rl-gcp"
KEY_FILE="/home/jovyan/jlewi-kubeflow-rl@kubeflow-dev.iam.gserviceaccount.com.json"
APP_DIR=os.path.join(ROOT_DIR, "rl-app")
KSBIN=os.path.join("/home/jovyan/ks_0.8.0_linux_amd64/ks")

IMAGE="gcr.io/kubeflow-dev/agents-ppo:cpu-cae5b3af"
# ksonnet env to use
ENV="jlewi-dev"

# The hostname used for Kubeflow
HOSTNAME="dev.kubeflow.org"

In [48]:
# GCP project to use
#
# settings for project: kubeflow-rl
#PROJECT="kubeflow-rl"
# Bucket to use
#BUCKET=PROJECT+"-kf"
# K8s cluster to use
#CLUSTER="kubeflow"
#ZONE="us-east1-d"
#NAMESPACE="rl"
# Root directory for the kubeflow-rl repository
#ROOT_DIR = "/home/jovyan/git_kubeflow-rl"
#SECRET_NAME = "kubeflow-rl-gcp"
#KEY_FILE="/home/jovyan/kubeflow-rl-f673306814d4.json"
IMAGE="gcr.io/kubeflow-rl/agents-ppo:cpu-cae5b3af"

You will need GCP credentials to access the cluster and GCP resources
  * Create a service account and download the private key
  * Use JupyterLab to upload the service account to your pod
  * Set the path to your service account in the cell below and then execute it to activate the service account

In [8]:
!gcloud auth activate-service-account --key-file={KEY_FILE}

Activated service account credentials for: [jlewi-kubeflow-rl@kubeflow-dev.iam.gserviceaccount.com]


In [9]:
!gsutil mb -p {PROJECT} gs://{BUCKET}

Creating gs://kubeflow-dev-jlewi/...


Get credentials for your cluster

In [12]:
!gcloud container clusters --project={PROJECT} --zone={ZONE} get-credentials {CLUSTER}

Fetching cluster endpoint and auth data.
kubeconfig entry generated for gpu-cluster.


Create the namespace

In [13]:
!kubectl create namespace {NAMESPACE}

namespace "jlewi-rl" created


Download and install ksonnet if needed

In [34]:
# For this to work you first need to use kubectl exec -ti to login as root
# You can than do chmod a+rw /usr/local/bin
!wget -O ~/ks_0.8.0_linux_amd64.tar.gz --quiet https://github.com/ksonnet/ksonnet/releases/download/v0.8.0/ks_0.8.0_linux_amd64.tar.gz
!tar -C ~/ -zvxf ~/ks_0.8.0_linux_amd64.tar.gz 

# Move ks appears to cause an exec format error when we run it.

#!mv ks_0.8.0_linux_amd64/ks /usr/local/bin/ks
#!chmod a+x /usr/local/bin/ks

ks_0.8.0_linux_amd64/LICENSE
ks_0.8.0_linux_amd64/README.md
ks_0.8.0_linux_amd64/CHANGELOG.md
ks_0.8.0_linux_amd64/CODE-OF-CONDUCT.md
ks_0.8.0_linux_amd64/CONTRIBUTING.md
ks_0.8.0_linux_amd64/ks


If running on GCP (or possibly another Cloud) you probably need to create a key with credentials to use for your job

In [14]:
SECRET_FILE_NAME="secret.json"
!kubectl create -n {NAMESPACE} secret generic {SECRET_NAME}  --from-file={SECRET_FILE_NAME}={KEY_FILE}

secret "kubeflow-rl-gcp" created


### Training

The objective of the training phase is to learn the parameterization of our model that confers a high level of performance on the provided task. Here we'll launch and monitor a job.

In [26]:
# Change to the ksonnet directory
os.chdir(os.path.join(APP_DIR))

#### Launching the TFJob

We'll use [ksonnet](https://ksonnet.io/) to parameterize and apply a TFJob configuration (i.e. run a job). Here you can change the image to be a custom job image, such as one built and deployed with build.sh, or use the one provided here if you only want to change parameters. Below we'll display the templated job YAML for reference.

In [38]:
# Check your cluster and see if that matches one of the existing ksonnet environments
# You want the kubernetes master server to be the same as the server listed for the ks environment
!kubectl cluster-info
!{KSBIN} env list

Kubernetes master is running at https://35.184.183.224
GLBCDefaultBackend is running at https://35.184.183.224/api/v1/namespaces/kube-system/services/default-http-backend/proxy
Heapster is running at https://35.184.183.224/api/v1/namespaces/kube-system/services/heapster/proxy
KubeDNS is running at https://35.184.183.224/api/v1/namespaces/kube-system/services/kube-dns/proxy
kubernetes-dashboard is running at https://35.184.183.224/api/v1/namespaces/kube-system/services/kubernetes-dashboard/proxy
Metrics-server is running at https://35.184.183.224/api/v1/namespaces/kube-system/services/metrics-server/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.
NAME     NAMESPACE SERVER
====     ========= ======
default            https://35.185.119.177
gke      rl        https://35.196.10.29
iap-test iap-test  https://35.196.10.29


In [41]:
# Create the environment if needed
!{KSBIN} env add {ENV}

INFO  Using context 'gke_kubeflow-dev_us-central1-a_gpu-cluster' from the kubeconfig file specified at the environment variable $KUBECONFIG
INFO  Creating environment 'jlewi-dev' with namespace '', pointing at server at address 'https://35.184.183.224'
INFO  Generating environment metadata at path '/home/jovyan/git_kubeflow-rl/rl-app/environments/jlewi-dev'
INFO  Environment 'jlewi-dev' pointing to namespace '' and server address at 'https://35.184.183.224' successfully created


In [50]:
import datetime
import uuid
import os

os.chdir(os.path.join(ROOT_DIR, "rl-app"))

HPARAM_SET="pybullet-kuka-ff"

now=datetime.datetime.now()

JOB_SALT=now.strftime("%m%d-%H%M") + "-" + uuid.uuid4().hex[0:4]
JOB_NAME=HPARAM_SET + "-" + JOB_SALT
LOG_DIR="gs://{0}/jobs/{1}".format(BUCKET, JOB_NAME)

!{KSBIN} param set --env={ENV} agents-ppo namespace {NAMESPACE}
!{KSBIN} param set --env={ENV} agents-ppo gcp_project {PROJECT}
!{KSBIN} param set --env={ENV} agents-ppo image {IMAGE}
# Dev cluster only has 4 CPU machine's; ideally would like to use 30 CPUs
!{KSBIN} param set --env={ENV} agents-ppo num_cpu 2
!{KSBIN} param set --env={ENV} agents-ppo job_tag {JOB_SALT}
!{KSBIN} param set --env={ENV} agents-ppo log_dir {LOG_DIR}
!{KSBIN} param set --env={ENV} agents-ppo name {JOB_NAME}
!{KSBIN} param set --env={ENV} agents-ppo gcp_secret {SECRET_NAME}
!{KSBIN} param set --env={ENV} agents-ppo secret_file_name {SECRET_FILE_NAME}
!{KSBIN} show {ENV} -c agents-ppo

!{KSBIN} apply {ENV} -c agents-ppo

INFO  Parameter 'namespace' successfully set to '"jlewi-rl"' for component 'agents-ppo' in environment 'jlewi-dev'
INFO  Parameter 'gcp_project' successfully set to '"kubeflow-dev"' for component 'agents-ppo' in environment 'jlewi-dev'
INFO  Parameter 'image' successfully set to '"gcr.io/kubeflow-dev/agents-ppo:cpu-cae5b3af"' for component 'agents-ppo' in environment 'jlewi-dev'
INFO  Parameter 'num_cpu' successfully set to '2' for component 'agents-ppo' in environment 'jlewi-dev'
INFO  Parameter 'job_tag' successfully set to '"0218-0117-352d"' for component 'agents-ppo' in environment 'jlewi-dev'
INFO  Parameter 'log_dir' successfully set to '"gs://kubeflow-dev-jlewi/jobs/pybullet-kuka-ff-0218-0117-352d"' for component 'agents-ppo' in environment 'jlewi-dev'
INFO  Parameter 'name' successfully set to '"pybullet-kuka-ff-0218-0117-352d"' for component 'agents-ppo' in environment 'jlewi-dev'
INFO  Parameter 'gcp_secret' successfully set to '"kubeflow-rl-gcp"' for component 'agents-ppo' i

Now we can list tfjobs and see that a job has been created.

In [55]:
!kubectl get tfjobs.kubeflow.org -n {NAMESPACE} -o yaml {JOB_NAME}

apiVersion: kubeflow.org/v1alpha1
kind: TFJob
metadata:
  clusterName: ""
  creationTimestamp: 2018-02-18T01:17:27Z
  generation: 0
  name: pybullet-kuka-ff-0218-0117-352d
  namespace: jlewi-rl
  resourceVersion: "751936"
  selfLink: /apis/kubeflow.org/v1alpha1/namespaces/jlewi-rl/tfjobs/pybullet-kuka-ff-0218-0117-352d
  uid: 7be8e091-1449-11e8-bc47-42010a800335
spec:
  RuntimeId: 7rao
  replicaSpecs:
  - replicas: 1
    template:
      metadata:
        creationTimestamp: null
      spec:
        containers:
        - args:
          - --logdir=gs://kubeflow-dev-jlewi/jobs/pybullet-kuka-ff-0218-0117-352d
          - --config=pybullet_kuka_ff
          - --network=feed_forward_gaussian
          - --policy_layers=200,100
          - --value_layers=200,100
          - --num_agents=30
          - --steps=10000000
          - --discount=0.995
          - --kl_target=0.01
          - --kl_cutoff_factor=2
          - --kl_cutoff_coef=1000
          - --algorithm=agents.ppo.PPOAlgorithm
    

#### Monitoring training

The IDs, status, and other metadata of pods involved in the training job can be displayed using the following:

In [56]:
!kubectl get pods -n {NAMESPACE} --show-all

NAME                                                  READY     STATUS    RESTARTS   AGE
pybullet-kuka-ff-0218-0117-352d-master-7rao-0-ckdfm   1/1       Running   0          6m
pybullet-kuka-ff-0218-0117-352d-tb-5c79676649-88tll   1/1       Running   0          4m


Logs from a specific pod can be displayed with the following (or streamed by adding the --follow flag):

In [57]:
import subprocess
master_pod = subprocess.check_output(["kubectl", "-n", NAMESPACE, "get", "pods", "--selector=tf_job_name=" + JOB_NAME,
                                      "-o", "jsonpath='{.items[*].metadata.name}'"]).decode("utf-8")
print(master_pod)
!kubectl -n {NAMESPACE} get pods -o yaml {master_pod}

'pybullet-kuka-ff-0218-0117-352d-master-7rao-0-ckdfm'
apiVersion: v1
kind: Pod
metadata:
  creationTimestamp: 2018-02-18T01:21:45Z
  generateName: pybullet-kuka-ff-0218-0117-352d-master-7rao-0-
  labels:
    controller-uid: 7bf06e37-1449-11e8-bc47-42010a800335
    job-name: pybullet-kuka-ff-0218-0117-352d-master-7rao-0
    job_type: MASTER
    kubeflow.org: ""
    runtime_id: 7rao
    task_index: "0"
    tf_job_name: pybullet-kuka-ff-0218-0117-352d
  name: pybullet-kuka-ff-0218-0117-352d-master-7rao-0-ckdfm
  namespace: jlewi-rl
  ownerReferences:
  - apiVersion: batch/v1
    blockOwnerDeletion: true
    controller: true
    kind: Job
    name: pybullet-kuka-ff-0218-0117-352d-master-7rao-0
    uid: 7bf06e37-1449-11e8-bc47-42010a800335
  resourceVersion: "752353"
  selfLink: /api/v1/namespaces/jlewi-rl/pods/pybullet-kuka-ff-0218-0117-352d-master-7rao-0-ckdfm
  uid: 155fcd52-144a-11e8-bc47-42010a800335
spec:
  containers:
  - args:
    - --logdir=gs://kubeflow-dev-jlewi/jobs/pybullet-kuk

In [58]:
!kubectl logs -n {NAMESPACE} {master_pod}

INFO:tensorflow:Tensorflow version: 1.3.0
INFO:tensorflow:Tensorflow git version: v1.3.0-rc2-20-g0787eee
INFO:tensorflow:Start a new run and write summaries and checkpoints to gs://kubeflow-dev-jlewi/jobs/pybullet-kuka-ff-0218-0117-352d.
INFO:tensorflow:Graph contains 44607 trainable variables.
2018-02-18 01:23:14.844325: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-18 01:23:14.844458: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-02-18 01:23:14.844470: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
INFO:tensorflow:
------------------

## Launching tensorboard
You can run tensorboard in your job as follows

In [54]:
!{KSBIN} param set --env={ENV} tensorboard name {JOB_NAME}
!{KSBIN} param set --env={ENV} tensorboard namespace {NAMESPACE}
!{KSBIN} param set --env={ENV} tensorboard log_dir {LOG_DIR}
!{KSBIN} param set --env={ENV} tensorboard secret {SECRET_NAME}
!{KSBIN} param set --env={ENV} tensorboard secret_file_name {SECRET_FILE_NAME}

print("Tensorboard will be available at:")
print("https://" + HOSTNAME + "/tensorboard/" + JOB_NAME+"/")

!{KSBIN} show {ENV} -c tensorboard


!{KSBIN} apply {ENV} -c tensorboard

INFO  Parameter 'name' successfully set to '"pybullet-kuka-ff-0218-0117-352d"' for component 'tensorboard' in environment 'jlewi-dev'
INFO  Parameter 'namespace' successfully set to '"jlewi-rl"' for component 'tensorboard' in environment 'jlewi-dev'
INFO  Parameter 'log_dir' successfully set to '"gs://kubeflow-dev-jlewi/jobs/pybullet-kuka-ff-0218-0117-352d"' for component 'tensorboard' in environment 'jlewi-dev'
INFO  Parameter 'secret' successfully set to '"kubeflow-rl-gcp"' for component 'tensorboard' in environment 'jlewi-dev'
INFO  Parameter 'secret_file_name' successfully set to '"secret.json"' for component 'tensorboard' in environment 'jlewi-dev'
---
apiVersion: apps/v1beta1
kind: Deployment
metadata:
  name: pybullet-kuka-ff-0218-0117-352d-tb
  namespace: jlewi-rl
spec:
  replicas: 1
  template:
    metadata:
      labels:
        app: tensorboard
        tb-job: pybullet-kuka-ff-0218-0117-352d
      name: pybullet-kuka-ff-0218-0117-352d
      namespace: jlewi-rl
    spec:
    

### Connecting to Tensorboard
To connect to tensorboard use `kubectl proxy` and then access it and the url given by the URL returned by evaluating the next cell

In [ ]:
PROXY_PORT=8001
url=print("http://127.0.0.1:{proxy_port}/api/v1/proxy/namespaces/{namespace}/services/{service_name}:80/".format(
    proxy_port=PROXY_PORT, namespace=NAMESPACE, service_name=JOB_NAME + "-tb"))
print(url)

#### Deleting jobs

In [3]:
!kubectl delete tfjobs -n {NAMESPACE} {JOB_NAME}

tfjob "pybullet-kuka-ff-0e90193e" deleted


### Rendering the model

When the job is complete there will be a subdirectory of the log dir named "render" with a number of short videos of episodes of the agent performing the grasping task. Here's an example of what one of those looks like in a well-trained model.

In [9]:
import io
import base64
from IPython.display import HTML

# Replace with the 
mp4_path = 'render.mp4'

video = io.open(mp4_path, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

### Great job! 🎉🎉🎉

If this is your first time working with these technologies you might be interested in some suggestions of good next steps. Here are some ideas:
- Try training with some other learning environments and tweet your results!
- Take a shot at implementing your own gym learning environment and repeat the above.